В данном ноутбуке представлен пример разметки текста для моделей TENER и SBERT. Для начала установим необходимые библиотеки

In [1]:
!git clone https://github.com/dialogue-evaluation/RuREBus # Клонирование репозитория RuREBus

Cloning into 'RuREBus'...
remote: Enumerating objects: 247, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 247 (delta 16), reused 4 (delta 1), pack-reused 217
Receiving objects: 100% (247/247), 14.22 MiB | 20.93 MiB/s, done.
Resolving deltas: 100% (118/118), done.


In [2]:
! wget https://storage.yandexcloud.net/natasha-navec/packs/navec_hudlit_v1_12B_500K_300d_100q.tar # Установка navec - words embedding 

--2023-01-26 13:14:13--  https://storage.yandexcloud.net/natasha-navec/packs/navec_hudlit_v1_12B_500K_300d_100q.tar
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 53012480 (51M) [application/x-tar]
Saving to: ‘navec_hudlit_v1_12B_500K_300d_100q.tar’

navec_hudlit_v1_12B 100%[===================>]  50.56M  16.1MB/s    in 4.0s    

2023-01-26 13:14:18 (12.5 MB/s) - ‘navec_hudlit_v1_12B_500K_300d_100q.tar’ saved [53012480/53012480]



In [5]:
# Установка библиотек для работы с words embedding
! pip install Navec 
! pip install slovnet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
# Распаковка текстов от RuREBus

!mkdir /content/train
!mkdir /content/test
!mkdir /content/test/test_part_1

!unzip /content/RuREBus/train_data/train_part_1.zip -d /content/train
!unzip /content/RuREBus/train_data/train_part_2.zip -d /content/train
!unzip /content/RuREBus/train_data/train_part_3.zip -d /content/train

!unzip /content/RuREBus/test_data/test_ner_only.zip -d /content/test/test_part_1


Archive:  /content/RuREBus/train_data/train_part_1.zip
   creating: /content/train/train_part_1/
  inflating: /content/train/train_part_1/.stats_cache  
  inflating: /content/train/train_part_1/20336081161101050428001_17_part_0.ann  
  inflating: /content/train/train_part_1/20336081161101050428001_17_part_0.txt  
  inflating: /content/train/train_part_1/20336081161101050428001_17_part_1.ann  
  inflating: /content/train/train_part_1/20336081161101050428001_17_part_1.txt  
  inflating: /content/train/train_part_1/20336081161101050428001_17_part_2.ann  
  inflating: /content/train/train_part_1/20336081161101050428001_17_part_2.txt  
  inflating: /content/train/train_part_1/20336241021100524345002_22_part_0.ann  
  inflating: /content/train/train_part_1/20336241021100524345002_22_part_0.txt  
  inflating: /content/train/train_part_1/20336241021100524345002_22_part_1.ann  
  inflating: /content/train/train_part_1/20336241021100524345002_22_part_1.txt  
  inflating: /content/train/train_par

In [6]:
# Установка библиотек для преобработки слов
! wget https://rusvectores.org/static/models/udpipe_syntagrus.model 
! pip install ufal.udpipe
! pip install 'gensim==3.7.0'

--2023-01-26 13:15:42--  https://rusvectores.org/static/models/udpipe_syntagrus.model
Resolving rusvectores.org (rusvectores.org)... 172.104.228.108
Connecting to rusvectores.org (rusvectores.org)|172.104.228.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 40616122 (39M)
Saving to: ‘udpipe_syntagrus.model’

udpipe_syntagrus.mo 100%[===================>]  38.73M  11.9MB/s    in 3.3s    

2023-01-26 13:15:46 (11.9 MB/s) - ‘udpipe_syntagrus.model’ saved [40616122/40616122]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 848.5/848.5 KB 22.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.4/23.4 MB 55.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gensim: filename=gensim-3.7.0-cp38-cp38-linux_x86_64.whl size=26637957 sh

In [7]:
! pip install "fastNLP==0.5" # Установка fastNLP 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.2/270.2 KB 9.8 MB/s eta 0:00:00


In [8]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [9]:
import os
import time
from datetime import datetime, timedelta

import numpy as np
import torch
import torch.nn as nn

try:
    from tqdm.auto import tqdm
except:
    from fastNLP.core.utils import _pseudo_tqdm as tqdm
import warnings

from fastNLP.core.batch import DataSetIter, BatchIter
from fastNLP.core.callback import CallbackManager, CallbackException, Callback
from fastNLP.core.dataset import DataSet
from fastNLP.core.losses import _prepare_losser
from fastNLP.core.metrics import _prepare_metrics
from fastNLP.core.optimizer import Optimizer
from fastNLP.core.sampler import Sampler
from fastNLP.core.sampler import RandomSampler
from fastNLP.core.tester import Tester
from fastNLP.core.utils import _CheckError
from fastNLP.core.utils import _build_args
from fastNLP.core.utils import _check_forward_error
from fastNLP.core.utils import _check_loss_evaluate
from fastNLP.core.utils import _move_dict_value_to_device
from fastNLP.core.utils import _get_func_signature
from fastNLP.core.utils import _get_model_device
from fastNLP.core.utils import _move_model_to_device
from fastNLP.core._parallel_utils import _model_contains_inner_module
from fastNLP.core._logger import logger

import codecs

In [10]:
import copy
import fastNLP
from fastNLP.embeddings import CNNCharEmbedding
from fastNLP.embeddings import StaticEmbedding, StackEmbedding

from subprocess import getoutput
getoutput("git clone -l -s https://github.com/fastnlp/TENER")

from fastNLP.embeddings import CNNCharEmbedding
from fastNLP import cache_results
from fastNLP import Trainer, GradientClipCallback, WarmupCallback
from torch import optim
from fastNLP import SpanFPreRecMetric, BucketSampler
from fastNLP.io.pipe.conll import OntoNotesNERPipe
from fastNLP.embeddings import StaticEmbedding, StackEmbedding, LSTMCharEmbedding
from TENER.modules.TransformerEmbedding import TransformerCharEmbed

from TENER.modules.callbacks import EvaluateCallback

from fastNLP.modules import ConditionalRandomField, allowed_transitions
from TENER.modules.transformer import TransformerEncoder

from torch import nn
import torch
import torch.nn.functional as F

from navec import Navec
import argparse

from collections import Counter
from fastNLP.io.pipe import Conll2003NERPipe

import pickle
import json

In [13]:
def clean_token(token, misc):
    out_token = token.strip().replace(' ', '')
    if token == 'Файл' and 'SpaceAfter=No' in misc:
        return None
    return out_token

Слудеющая функция обрабатывает текст, можно выделить часть речи или убрать пунктуацию. На вход подается строка и pipeline

In [15]:
def process1(pipeline, text='Строка', keep_pos=True, keep_punct=False):
    entities = {'PROPN'}
    named = False
    memory = []
    mem_case = None
    mem_number = None
    tagged_propn = []

    # обрабатываем текст, получаем результат в формате conllu:
    processed = pipeline.process(text)
    

    # пропускаем строки со служебной информацией:
    content = [l for l in processed.split('\n') if not l.startswith('#')]

    # извлекаем из обработанного текста токены
    tagged = [w.split('\t') for w in content if w]

    for t in tagged:
        if len(t) != 10:
            continue
        (word_id, token, lemma, pos, xpos, feats, head, deprel, deps, misc) = t
        
        token = clean_token(token, misc)
        tagged_propn.append(token.lower()+'_'+pos)
        
    if not keep_punct: 
        tagged_propn = [word for word in tagged_propn if word.split('_')[1] != 'PUNCT']

    if not keep_pos:
        tagged_propn = [word.split('_')[0] for word in tagged_propn]
        
    return tagged_propn

Следующая функция также принимает строку и  параметры, которые затем обновляются и будут использоваться для создания датасета.

In [16]:
from ufal.udpipe import Model, Pipeline
import os
import re
import sys
import gensim

def tag_ud(dictionary, all_words, instances, char_id, text='Текст нужно передать функции в виде строки!', 
           model='model'):
  
    """
    Parameters:
    dictionary - список tuple от уникальных слов всех текстов и индекса navec
    all_words - список слов из всех текстов
    instances - словарь, который будет использоваться в дальнейшем для создания датасета
    char_id - словарь с диапазоном символов, который занимает строка
    text - строка
    model - модель для pipeline

    Return:
    возвращаются расширенные dictionary, all_words, instances, char_id
    """
    process_pipeline = Pipeline(model, 'tokenize', Pipeline.DEFAULT, Pipeline.DEFAULT, 'conllu')
    ind, ind2 = 0, 0
    
    for line in text:
        
        output = process1(process_pipeline, text=line, keep_pos=False, keep_punct=False) # Обработка строки
        
        if len(output) == 0:
          continue

        ords = [] # список содержащий индексы, которые будут подаваться в embedding
        
        ind2 += len([*line])
        char_id.append((ind, ind2)) # добавление диапазона
        ind = ind2

        for word in output:
          
          # Проверка наличия слова в navec и извлечение индекса
          no_unk_flag = False 
          
          if word in navec:
            id = navec.vocab[word]
            no_unk_flag = True
          else: id = navec.vocab['<unk>']

          ords.append(id)

          all_words.append(word) # добавление слова
          if word in dictionary or no_unk_flag == False: continue
          dictionary.append((word, id)) # добавление tuple (слово, индекс)

        # Добавление данных
        instances['sentence'].append(output)
        instances['ords'].append(ords)
        instances['seq_len'].append(len(ords))
        instances['label'].append([0] * len(ords))
        instances['flag'].append([True] *len(ords))
        
       
        
    return dictionary, all_words, instances, char_id

In [17]:
from fastNLP import Padder, EngChar2DPadder, SequentialSampler, AutoPadder
from fastNLP import Instance, DataSet
from navec import Navec
# загрузка Navec
path = '/content/navec_hudlit_v1_12B_500K_300d_100q.tar'
navec = Navec.load(path)

Следубщая функция возвращает списки путей к файлам text и ann, принимая на вход ссылку на исходную папку. Исходная папка должна содежать еще одни папки с нужнымы файлами 

In [18]:
def txt_ann_file_paths(path):

  txt_path_spisok = []

  for folder in os.listdir(path):

    folder_path = os.path.join(path, folder)
    for f in os.listdir(folder_path):
      if 'txt' in f:
        file_txt_path = os.path.join(folder_path, f)
        txt_path_spisok.append(file_txt_path)

  ann_path_spisok = []

  for path in txt_path_spisok:
    ann_path_spisok.append(path.replace('txt', 'ann'))

  return txt_path_spisok, ann_path_spisok


In [19]:
def create_instances_for_dataset(txt_path_spisok, dictionary=None, all_words=None, char_id=None, instances=None):
  
  #Функция, которая работает с tag_ud - инициализирует переменные и модель для pipline, делит текст на строки

  count = 0 # Количество текстов
  model = Model.load('/content/udpipe_syntagrus.model')
  if dictionary==None or all_words==None or char_id==None or instances==None:
    dictionary = []
    all_words = []
    char_id = dict()
    instances = dict(sentence=[], ords=[], label=[], seq_len=[], flag= [])

  for path in txt_path_spisok:

    print(f'Documents are downloaded {count} of {len(txt_path_spisok)}')
    char_id[str(count)] = []
    

    with open(path, 'r') as text:
      text = text.readlines()
      
      dictionary, all_words, instances, char_id[str(count)] = tag_ud(dictionary=dictionary, all_words=all_words, 
                                                        instances=instances, char_id=char_id[str(count)],
                                                        text=text, model=model)
    count += 1

  return dictionary, all_words, instances, char_id

Следующая функция извлекает данные из аннтоаций, которые будут использоваться при дальнейшей разметке

In [206]:
def create_spans(ann_path_spisok):

  class_spisok = [] # Список всех классов во всех ann
  spans = dict() # словарь который будет содержать разметку для каждого txt файла
  md = Model.load('/content/udpipe_syntagrus.model')

  for i, path in enumerate(ann_path_spisok):
    print(i)
    with open(path, 'r') as f:

      lines = f.readlines()
      span = []
      for line in lines:
     
        process_pipeline = Pipeline(md, 'tokenize', Pipeline.DEFAULT, Pipeline.DEFAULT, 'conllu') 
        l = process1(process_pipeline, text=line, keep_pos=False, keep_punct=False) # такая же предобработка строки ann, как и txt

        # Добавление только NER классов, слов и их диапазона
        if 'r' in l[0]:
          continue

        span_start =int(l[2])
        span_end =int(l[3])
        
        cl = l[1].upper()

        span.append([span_start, span_end, cl] + l[4:])
        class_spisok.append(cl)

    span = sorted(span)
    spans[str(i)] = span

  return spans, class_spisok

In [163]:
train_path = '/content/train'
test_path = '/content/test'

# Создание списков train и test списков с ann и txt файлами
train_txt_path_spisok, train_ann_path_spisok = txt_ann_file_paths(train_path)
test_txt_path_spisok, test_ann_path_spisok = txt_ann_file_paths(test_path)

# Создание необходимых переменных, описанных в tag_ud
train_dictionary, train_all_words, train_instances, train_char_id = create_instances_for_dataset(sorted(train_txt_path_spisok))
test_dictionary, test_all_words, test_instances, test_char_id = create_instances_for_dataset(sorted(test_txt_path_spisok))

# Создание диапазонов из ann файлов
train_spans, train_class_spisok_number = create_spans(sorted(train_ann_path_spisok))
test_spans, test_class_spisok_number = create_spans(sorted(test_ann_path_spisok))

Documents are downloaded 0 of 188
Documents are downloaded 1 of 188
Documents are downloaded 2 of 188
Documents are downloaded 3 of 188
Documents are downloaded 4 of 188
Documents are downloaded 5 of 188
Documents are downloaded 6 of 188
Documents are downloaded 7 of 188
Documents are downloaded 8 of 188
Documents are downloaded 9 of 188
Documents are downloaded 10 of 188
Documents are downloaded 11 of 188
Documents are downloaded 12 of 188
Documents are downloaded 13 of 188
Documents are downloaded 14 of 188
Documents are downloaded 15 of 188
Documents are downloaded 16 of 188
Documents are downloaded 17 of 188
Documents are downloaded 18 of 188
Documents are downloaded 19 of 188
Documents are downloaded 20 of 188
Documents are downloaded 21 of 188
Documents are downloaded 22 of 188
Documents are downloaded 23 of 188
Documents are downloaded 24 of 188
Documents are downloaded 25 of 188
Documents are downloaded 26 of 188
Documents are downloaded 27 of 188
Documents are downloaded 28 of

In [366]:
# Создание словарей из индекса в слова
class2idx = dict(O=0, MET=1, ECO=2, BIN=3, CMP=4, QUA=5, ACT=6, INST=7, SOC=8)
idx2class = {index:word for word, index in zip(class2idx.keys(), class2idx.values())}

Следующая функция проставляет метки для словарей 

In [338]:
def create_labels(class2idx, spans, char_id, instances):

  def change_label(id, idx):
    # Изменение label, которое было получено при инициализации и смена флага
    # id - индекс строки, idx - индекс слова 
    cl = class2idx[s[2]]
    add_condition = instances['flag'][id][idx]

    if add_condition:
      sp.append(0)
      instances['label'][id][idx] += cl
      instances['flag'][id][idx] = False

  instances = copy.deepcopy(instances)
  add_to_i = 0 
  sp = []
  for c in range(len(spans.keys())): # итерация по spans каждого ann документа
    for s in spans[str(c)]: # всех Итерация span в одном документе
      char_id_text = char_id[str(c)] 
      for i, idd in enumerate(char_id_text): # итерация по диапазонам каждой строки txt документа

        start, end = idd
        
        id = i + add_to_i
        # Проверка входит ли диапазон из ann в диапазон строки из txt
        if start <= s[0] <= end and start<= s[1] <= end + 2:
          sentence = instances['sentence'][id]  
          
          for w in s[3:]:

            # Иногда строка span может "пройти" мимо диапазона текста или нужные слова отличаются по написанию в txt и ann      
            if w not in sentence:
              
              for back_step in range(1, id+1):

                new_id = id - back_step
                new_sentence = instances['sentence'][new_id]

                if w in instances['sentence'][new_id]:
                  idx = new_sentence.index(w.lower())

                  # Вызов смены индекса
                  change_label(new_id, idx)
                  break

                if new_id == 0 or back_step>1:
                  break 
              
              continue
            
            idx = sentence.index(w.lower())
            
            change_label(id, idx)

          if s == spans[str(c)][-1]:
            add_to_i += len(char_id_text)

          break
  

  return instances

In [334]:
# Создание spans для будущей разметки
train_spans, train_class_spisok_number = create_spans(sorted(train_ann_path_spisok))
test_spans, test_class_spisok_number = create_spans(sorted(test_ann_path_spisok))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [368]:
# Разметка по классам
train_instances = create_labels(class2idx, train_spans, train_char_id, train_instances)
test_instances = create_labels(class2idx, test_spans, test_char_id, test_instances)

In [370]:
# Сохранение на диск параметров, нужных для будущего датасета

train_json_string = train_instances

test_json_string = test_instances

with open('/content/gdrive/MyDrive/3_4_lab/TENER/Last_instances_json/Train/train_instances.json', 'w') as f:
  json.dump(train_json_string, f)

with open('/content/gdrive/MyDrive/3_4_lab/TENER/Last_instances_json/Train/train_class_spisok_number.json', 'w') as f:
  json.dump(train_class_spisok_number, f)

with open('/content/gdrive/MyDrive/3_4_lab/TENER/Last_instances_json/Train/train_all_words.json', 'w') as f:
  json.dump(train_all_words, f)

with open('/content/gdrive/MyDrive/3_4_lab/TENER/Last_instances_json/Train/train_dictionary.json', 'w') as f:
  json.dump(train_dictionary, f)


with open('/content/gdrive/MyDrive/3_4_lab/TENER/Last_instances_json/Test/test_instances.json', 'w') as f:
  json.dump(test_json_string, f)

with open('/content/gdrive/MyDrive/3_4_lab/TENER/Last_instances_json/Test/test_class_spisok_number.json', 'w') as f:
  json.dump(test_class_spisok_number, f)

with open('/content/gdrive/MyDrive/3_4_lab/TENER/Last_instances_json/Test/test_all_words.json', 'w') as f:
  json.dump(test_all_words, f)

with open('/content/gdrive/MyDrive/3_4_lab/TENER/Last_instances_json/Test/test_dictionary.json', 'w') as f:
  json.dump(test_dictionary, f)

In [371]:
# Инициализация параметров, loaded_train_instances, loaded_test_instances используются сразу в других ноутбуках
filepath = '/content/gdrive/MyDrive/3_4_lab/TENER/Last_instances_json/Train/train_instances.json'
with open(filepath, encoding='utf-8') as f:
  loaded_train_instances = json.load(f)

filepath = '/content/gdrive/MyDrive/3_4_lab/TENER/Last_instances_json/Train/train_class_spisok_number.json'
with open(filepath, encoding='utf-8') as f:
  train_class_spisok_number = json.load(f)

filepath = '/content/gdrive/MyDrive/3_4_lab/TENER/Last_instances_json/Train/train_all_words.json'
with open(filepath, encoding='utf-8') as f:
  loaded_train_all_words = json.load(f)

filepath = '/content/gdrive/MyDrive/3_4_lab/TENER/Last_instances_json/Train/train_dictionary.json'
with open(filepath, encoding='utf-8') as f:
  loaded_train_dictionary = json.load(f)


filepath = '/content/gdrive/MyDrive/3_4_lab/TENER/Last_instances_json/Test/test_instances.json'
with open(filepath, encoding='utf-8') as f:
  loaded_test_instances = json.load(f)

filepath = '/content/gdrive/MyDrive/3_4_lab/TENER/Last_instances_json/Test/test_class_spisok_number.json'
with open(filepath, encoding='utf-8') as f:
  test_class_spisok_number = json.load(f)

filepath = '/content/gdrive/MyDrive/3_4_lab/TENER/Last_instances_json/Test/test_all_words.json'
with open(filepath, encoding='utf-8') as f:
  loaded_test_all_words = json.load(f)

filepath = '/content/gdrive/MyDrive/3_4_lab/TENER/Last_instances_json/Test/test_dictionary.json'
with open(filepath, encoding='utf-8') as f:
  loaded_test_dictionary = json.load(f)


In [372]:
# Создание словарей класс - индекс, с переименованием класса - необходимо в некоторых библиотеках fastNLP
class2idx = dict(O=0, IMET=1, ECO=2, BIN=3, SMP=4, BUA=5, ICT=6, INST=7, SOC=8)
idx2class = {index:word for word, index in zip(class2idx.keys(), class2idx.values())}

train_class_O_number = len(loaded_train_all_words) - len(train_class_spisok_number)
test_class_O_number = len(loaded_test_all_words) - len(test_class_spisok_number)

train_class_spisok_number += ['O'] * train_class_O_number
test_class_spisok_number += ['O'] * test_class_O_number

train_class_freq = Counter(train_class_spisok_number)
test_class_freq = Counter(test_class_spisok_number)

In [373]:
# Создание datasets для тренировки и теста с использованием DataSet от fastNLP

dt_train = DataSet(loaded_train_instances)

dt_train.rename_field('ords', 'chars')
dt_train.rename_field('label', 'target')

dt_train.set_padder('chars', AutoPadder())
dt_train.set_input('chars', 'target')
dt_train.set_target('seq_len')
dt_train.set_pad_val('chars', 500001)


dt_test = DataSet(loaded_test_instances)

dt_test.rename_field('ords', 'chars')
dt_test.rename_field('label', 'target')

dt_test.set_padder('chars', AutoPadder())
dt_test.set_input('chars', 'target')
dt_test.set_target('seq_len', 'target')
dt_test.set_pad_val('chars', 500001)


+------------------+---------+--------------------+--------------------+--------------------+
| sentence         | seq_len | flag               | chars              | target             |
+------------------+---------+--------------------+--------------------+--------------------+
| ['докумен...     | 3       | [True, True, Tr... | [112995, 331177... | [0, 0, 0]          |
| ['админис...     | 3       | [True, True, Fa... | [16879, 62910, ... | [0, 0, 2]          |
| ['постано...     | 1       | [True]             | [324895]           | [0]                |
| ['от', '22', ... | 7       | [True, True, Tr... | [274784, 500000... | [0, 0, 0, 0, 0,... |
| ['о', 'бюдж...   | 5       | [True, True, Tr... | [252874, 51049,... | [0, 0, 0, 0, 0]... |
| ['на', 'пер...   | 5       | [True, True, Tr... | [225007, 296266... | [0, 0, 0, 0, 0]... |
| ['в', 'соот...   | 27      | [True, True, Tr... | [51199, 410755,... | [0, 0, 0, 0, 0,... |
| ['1', 'утве...   | 13      | [True, False, T... | [500000,

In [374]:
# Создание словаря слово - индекс и списка частоты слов для библиотеки CNNCharEmbedding
word2idx = dict()
word2idx['<pad>'] = 500001
word2idx['<unk>'] = 500000
for t in loaded_train_dictionary:

  word, id = t
  word2idx[word] = id

for t in loaded_test_dictionary:

  word, id = t
  word2idx[word] = id

idx2word = {index:word for word, index in zip(word2idx.keys(), word2idx.values())}
train_freq = Counter(loaded_train_all_words)

In [375]:

class voc:

  def __init__(self, word2idx, idx2word, word_count, max_size=None, min_freq=None, padding='<pad>', unknown='<unk>', rebuild=False, _no_create_word=Counter()):

    self._word2idx = word2idx
    self._idx2word = idx2word
    self.max_size = max_size
    self.min_freq = min_freq
    self.padding = padding
    self.unknown = unknown
    self.rebuild = rebuild
    self._no_create_word = _no_create_word
    self.word_count = word_count
    self.padding_idx = 0
    self.unknown_idx = 0

  def __iter__(self):
        
        for word, index in self._word2idx.items():
            yield self._idx2word[index], index

  def _is_word_no_create_entry(self, word):
        
        return word in self._no_create_word

  def __len__(self):
        return len(self._word2idx)


In [376]:
# упаковка словарей для CNNCharEmbedding
dict_for_embed = voc(word2idx=word2idx, idx2word=idx2word,word_count=train_freq)

In [377]:
# Библиотека CNNCharEmbedding от fastNLP, изменный для работы с Navec

from typing import List

import torch
import torch.nn as nn
import torch.nn.functional as F

from fastNLP.embeddings.embedding import TokenEmbedding
from fastNLP.embeddings.static_embedding import StaticEmbedding
from fastNLP.embeddings.utils import _construct_char_vocab_from_vocab
from fastNLP.embeddings.utils import get_embeddings
from fastNLP.core import logger
from fastNLP.core.vocabulary import Vocabulary

class CNNCharEmbedding(TokenEmbedding):
    r"""
    使用CNN生成character embedding。CNN的结构为, embed(x) -> Dropout(x) -> CNN(x) -> activation(x) -> pool -> fc -> Dropout.
    不同的kernel大小的fitler结果是concat起来然后通过一层fully connected layer, 然后输出word的表示。

    Example::

        >>> import torch
        >>> from fastNLP import Vocabulary
        >>> from fastNLP.embeddings import CNNCharEmbedding
        >>> vocab = Vocabulary().add_word_lst("The whether is good .".split())
        >>> embed = CNNCharEmbedding(vocab, embed_size=50)
        >>> words = torch.LongTensor([[vocab.to_index(word) for word in "The whether is good .".split()]])
        >>> outputs = embed(words)
        >>> outputs.size()
        >>> # torch.Size([1, 5，50])

    """
    
    def __init__(self, vocab: Vocabulary, embed_size: int = 50, char_emb_size: int = 50, word_dropout: float = 0,
                 dropout: float = 0, filter_nums: List[int] = (40, 30, 20), kernel_sizes: List[int] = (5, 3, 1),
                 pool_method: str = 'max', activation='relu', min_char_freq: int = 2, pre_train_char_embed: str = None,
                 requires_grad:bool=True, include_word_start_end:bool=True):
        
        super(CNNCharEmbedding, self).__init__(vocab, word_dropout=word_dropout, dropout=dropout)
        
        for kernel in kernel_sizes:
            assert kernel % 2 == 1, "Only odd kernel is allowed."
        
        assert pool_method in ('max', 'avg')
        self.pool_method = pool_method
        
        if isinstance(activation, str):
            if activation.lower() == 'relu':
                self.activation = F.relu
            elif activation.lower() == 'sigmoid':
                self.activation = F.sigmoid
            elif activation.lower() == 'tanh':
                self.activation = F.tanh
        elif activation is None:
            self.activation = lambda x: x
        elif callable(activation):
            self.activation = activation
        else:
            raise Exception(
                "Undefined activation function: choose from: [relu, tanh, sigmoid, or a callable function]")
        
        logger.info("Start constructing character vocabulary.")
        
        self.char_vocab = _construct_char_vocab_from_vocab(vocab, min_freq=min_char_freq,
                                                           include_word_start_end=include_word_start_end)
        self.char_pad_index = self.char_vocab.padding_idx
        
        logger.info(f"In total, there are {len(self.char_vocab)} distinct characters.")
        
        max_word_len = max(map(lambda x: len(x[0]), vocab))
        if include_word_start_end:
            max_word_len += 2
        
        self.register_buffer('words_to_chars_embedding', torch.full((500002, max_word_len),
                                                                fill_value=self.char_pad_index, dtype=torch.long))
        self.register_buffer('word_lengths', torch.zeros(500002).long())
        
        for word, index in vocab:
            
            if include_word_start_end:
                word = ['<bow>'] + list(word) + ['<eow>']
            
            self.words_to_chars_embedding[index, :len(word)] = torch.LongTensor([self.char_vocab.to_index(c) for c in word])
            self.word_lengths[index] = len(word)
        
        if pre_train_char_embed:
            self.char_embedding = StaticEmbedding(self.char_vocab, model_dir_or_name=pre_train_char_embed)
        else:
            self.char_embedding = get_embeddings((len(self.char_vocab), char_emb_size))
        
        self.convs = nn.ModuleList([nn.Conv1d(
            self.char_embedding.embedding_dim, filter_nums[i], kernel_size=kernel_sizes[i], bias=True,
            padding=kernel_sizes[i] // 2)
            for i in range(len(kernel_sizes))])
        self._embed_size = embed_size
        self.fc = nn.Linear(sum(filter_nums), embed_size)
        self.requires_grad = requires_grad

    def forward(self, words):
        r"""
        

        :param words: [batch_size, max_len]
        :return: [batch_size, max_len, embed_size]
        """
        words = self.drop_word(words)
        batch_size, max_len = words.size()
        chars = self.words_to_chars_embedding[words]  # batch_size x max_len x max_word_len
        word_lengths = self.word_lengths[words]  # batch_size x max_len
        max_word_len = word_lengths.max()
        chars = chars[:, :, :max_word_len]
        
        chars_masks = chars.eq(self.char_pad_index)  # batch_size x max_len x max_word_len 
        chars = self.char_embedding(chars)  # batch_size x max_len x max_word_len x embed_size
        chars = self.dropout(chars)
        reshaped_chars = chars.reshape(batch_size * max_len, max_word_len, -1)
        reshaped_chars = reshaped_chars.transpose(1, 2)  # B' x E x M
        conv_chars = [conv(reshaped_chars).transpose(1, 2).reshape(batch_size, max_len, max_word_len, -1)
                      for conv in self.convs]
        conv_chars = torch.cat(conv_chars, dim=-1).contiguous()  # B x max_len x max_word_len x sum(filters)
        conv_chars = self.activation(conv_chars)
        if self.pool_method == 'max':
            conv_chars = conv_chars.masked_fill(chars_masks.unsqueeze(-1), float('-inf'))
            chars, _ = torch.max(conv_chars, dim=-2)  # batch_size x max_len x sum(filters)
        else:
            conv_chars = conv_chars.masked_fill(chars_masks.unsqueeze(-1), 0)
            chars = torch.sum(conv_chars, dim=-2) / chars_masks.eq(False).sum(dim=-1, keepdim=True).float()
        chars = self.fc(chars)
        return self.dropout(chars)


In [378]:
char_embed = CNNCharEmbedding(vocab=dict_for_embed, embed_size=90, char_emb_size=90, 
                              filter_nums=[90], kernel_sizes=[3], word_dropout=0, 
                              dropout=0.1, pool_method='max', 
                              include_word_start_end=False, min_char_freq=2)

Start constructing character vocabulary.
In total, there are 62 distinct characters.


In [379]:
from torch.nn.modules.container import ModuleList
from slovnet.model.emb import NavecEmbedding
# Создание embedding для модели, использующий CNNCharEmbedding и Navec
class embed:

  def __init__(self, word_embed, char_embed, p=0.1):

    self.word_embed = nn.Sequential(word_embed, nn.Dropout(p=p))
    self.char_embed = char_embed
    
    self.embed = nn.ModuleList([self.word_embed, self.char_embed])

    self.embed_size = 330

  def forward(self, id):

    assert type(id) == torch.Tensor, 'emb only processes torch.Tensor'
    assert len(id.shape) == 2, f'emb only processes Tensor with 2 dimensions, your id.shape = {id.shape}'

    out = self.embed[0](id)
    for embed in self.embed[1:]:

      out = torch.cat((out, embed(id)), -1)

    return out

In [380]:
# Проверка работы embedding

word_embed = NavecEmbedding(navec)

embedding = embed(word_embed=word_embed, char_embed=char_embed)
count = 0
for id, y in DataSetIter(dt_train, sampler=SequentialSampler(), batch_size=2):

  words = id['chars']
  count += 1
  out = embedding.forward(words)
  if count == 100:
    break

count = 0
for id, y in DataSetIter(dt_test, sampler=SequentialSampler(), batch_size=2):

  words = id['chars']
  count += 1
  out = embedding.forward(words)

  if count == 100:
    break  

In [381]:
from fastNLP.core.vocabulary import Vocabulary

class tar(Vocabulary):

  def __init__(self, word2idx, idx2word, word_count, max_size=None, min_freq=None, 
               padding='<pad>', unknown='<unk>', rebuild=False, _no_create_word=Counter()):

    self._word2idx = word2idx
    self._idx2word = idx2word
    self.max_size = max_size
    self.min_freq = min_freq
    self.padding = padding
    self.unknown = unknown
    self.rebuild = rebuild
    self._no_create_word = _no_create_word
    self.word_count = word_count
    self.padding = None
    self.unknown = None

  def __iter__(self):
        
        for index in range(len(self._word2idx)):
            yield self._idx2word[index], index

  def _is_word_no_create_entry(self, word):
        
        return word in self._no_create_word

  def __len__(self):
        return len(self._word2idx)

  def items(self):
        return self._idx2word.items()

In [382]:
# упаковка меток и словарей 
tar_for_embed = tar(word2idx=class2idx, idx2word=idx2class,word_count=train_class_freq)

Дальше идут функции, необходимые для работы Trainer от fastNLP. Сам Trainer был немного изменен в некоторых строках для работы с Navec

In [385]:
DEFAULT_CHECK_BATCH_SIZE = 2
DEFAULT_CHECK_NUM_BATCH = 2


def _get_value_info(_dict):
    # given a dict value, return information about this dict's value. Return list of str
    strs = []
    for key, value in _dict.items():
        _str = ''
        if isinstance(value, torch.Tensor):
            _str += "\t{}: (1)type:torch.Tensor (2)dtype:{}, (3)shape:{} ".format(key,
                                                                                  value.dtype, value.size())
        elif isinstance(value, np.ndarray):
            _str += "\t{}: (1)type:numpy.ndarray (2)dtype:{}, (3)shape:{} ".format(key,
                                                                                   value.dtype, value.shape)
        else:
            _str += "\t{}: type:{}".format(key, type(value))
        strs.append(_str)
    return strs


from numbers import Number
from fastNLP.core.batch import _to_tensor


def _check_code(dataset, model, losser, metrics, forward_func, batch_size=DEFAULT_CHECK_BATCH_SIZE,
                dev_data=None, metric_key=None, check_level=0):
    # check get_loss 方法
    model_device = _get_model_device(model=model)
    def _iter():
        start_idx = 0
        while start_idx<len(dataset):
            batch_x = {}
            batch_y = {}
            for field_name, field in dataset.get_all_fields().items():
                indices = list(range(start_idx, min(start_idx+batch_size, len(dataset))))
                if field.is_target or field.is_input:
                    batch = field.get(indices)
                    if field.dtype is not None and \
                            issubclass(field.dtype, Number) and not isinstance(batch, torch.Tensor):
                        batch, _ = _to_tensor(batch, field.dtype)
                    if field.is_target:
                        batch_y[field_name] = batch
                    if field.is_input:
                        batch_x[field_name] = batch
            yield (batch_x, batch_y)
            start_idx += batch_size

    for batch_count, (batch_x, batch_y) in enumerate(_iter()):
        _move_dict_value_to_device(batch_x, batch_y, device=model_device)
        # forward check
        if batch_count == 0:
            info_str = ""
            input_fields = _get_value_info(batch_x)
            target_fields = _get_value_info(batch_y)
            if len(input_fields) > 0:
                info_str += "input fields after batch(if batch size is {}):\n".format(batch_size)
                info_str += "\n".join(input_fields)
                info_str += '\n'
            else:
                raise RuntimeError("There is no input field.")
            if len(target_fields) > 0:
                info_str += "target fields after batch(if batch size is {}):\n".format(batch_size)
                info_str += "\n".join(target_fields)
                info_str += '\n'
            else:
                info_str += 'There is no target field.'
            logger.info(info_str)
            _check_forward_error(forward_func=forward_func, dataset=dataset,
                                 batch_x=batch_x, check_level=check_level)
        refined_batch_x = _build_args(forward_func, **batch_x)
        pred_dict = model(**refined_batch_x)
        func_signature = _get_func_signature(forward_func)
        if not isinstance(pred_dict, dict):
            raise TypeError(f"The return value of {func_signature} should be `dict`, not `{type(pred_dict)}`.")
        
        # loss check
        try:
            loss = losser(pred_dict, batch_y)
            # check loss output
            if batch_count == 0:
                if not isinstance(loss, torch.Tensor):
                    raise TypeError(
                        f"The return value of {_get_func_signature(losser.get_loss)} should be `torch.Tensor`, "
                        f"but got `{type(loss)}`.")
                if len(loss.size()) != 0:
                    raise ValueError(
                        f"The size of return value of {_get_func_signature(losser.get_loss)} is {loss.size()}, "
                        f"should be torch.size([])")
            loss.backward()
        except _CheckError as e:
            # TODO: another error raised if _CheckError caught
            pre_func_signature = _get_func_signature(forward_func)
            _check_loss_evaluate(prev_func_signature=pre_func_signature, func_signature=e.func_signature,
                                 check_res=e.check_res, pred_dict=pred_dict, target_dict=batch_y,
                                 dataset=dataset, check_level=check_level)
        model.zero_grad()
        if batch_count + 1 >= DEFAULT_CHECK_NUM_BATCH:
            break
    
    if dev_data is not None:
        tester = Tester(data=dev_data[:batch_size * DEFAULT_CHECK_NUM_BATCH], model=model, metrics=metrics,
                        batch_size=batch_size, verbose=-1, use_tqdm=False)
        evaluate_results = tester.test()
        _check_eval_results(metrics=evaluate_results, metric_key=metric_key, metric_list=metrics)


def _check_eval_results(metrics, metric_key, metric_list):
    # metrics: tester返回的结果
    # metric_key: 一个用来做筛选的指标，来自Trainer的初始化
    # metric_list: 多个用来做评价的指标，来自Trainer的初始化
    if isinstance(metrics, tuple):
        loss, metrics = metrics
    
    if isinstance(metrics, dict):
        metric_dict = list(metrics.values())[0]  # 取第一个metric
        
        if metric_key is None:
            indicator_val, indicator = list(metric_dict.values())[0], list(metric_dict.keys())[0]
        else:
            # metric_key is set
            if metric_key not in metric_dict:
                raise RuntimeError(f"metric key {metric_key} not found in {metric_dict}")
            indicator_val = metric_dict[metric_key]
            indicator = metric_key
    else:
        raise RuntimeError("Invalid metrics type. Expect {}, got {}".format((tuple, dict), type(metrics)))
    return indicator, indicator_val

class Trainer(object):
    """
    Trainer在fastNLP中用于组织单任务的训练过程，可以避免用户在不同训练任务中重复撰写
        (1) epoch循环;
        (2) 将数据分成不同的Batch;
        (3) 对Batch进行pad;
        (4) 每个epoch结束或一定step后进行验证集验证;
        (5) 保存获得更好验证性能的模型等。
    
    详细的介绍参见 :mod:`fastNLP.core.trainer`
    """
    def __init__(self, train_data, model, optimizer=None, loss=None,
                 batch_size=32, sampler=None, drop_last=False, update_every=1,
                 num_workers=0, n_epochs=10, print_every=5,
                 dev_data=None, metrics=None, metric_key=None,
                 validate_every=-1, save_path=None, use_tqdm=True, device=None,
                 callbacks=None, check_code_level=0, **kwargs):
        """
        
        :param train_data: 训练集， :class:`~fastNLP.DataSet` 类型。
        :param nn.modules model: 待训练的模型
        :param optimizer: `torch.optim.Optimizer` 优化器。如果为None，则Trainer使用默认的Adam(model.parameters(), lr=4e-3)这个优化器
        :param int batch_size: 训练和验证的时候的batch大小。
        :param loss: 使用的 :class:`~fastNLP.core.losses.LossBase` 对象。当为None时，默认使用 :class:`~fastNLP.LossInForward`
        :param sampler: Batch数据生成的顺序， :class:`~fastNLP.Sampler` 类型。如果为None，默认使用 :class:`~fastNLP.RandomSampler`
        :param drop_last: 如果最后一个batch没有正好为batch_size这么多数据，就扔掉最后一个batch
        :param num_workers: int, 有多少个线程来进行数据pad处理。
        :param update_every: int, 多少步更新一次梯度。用于希望累计梯度的场景，比如需要128的batch_size, 但是直接设为128
            会导致内存不足，通过设置batch_size=32, update_every=4达到目的。当optimizer为None时，该参数无效。
        :param int n_epochs: 需要优化迭代多少次。
        :param int print_every: 多少次反向传播更新tqdm显示的loss; 如果use_tqdm=False, 则多少次反向传播打印loss。
        :param dev_data: 用于做验证的DataSet， :class:`~fastNLP.DataSet` 类型。
        :param metrics: 验证的评估函数。可以只使用一个 :class:`Metric<fastNLP.core.metrics.MetricBase>` ，
            也可以使用多个 :class:`Metric<fastNLP.core.metrics.MetricBase>` ，通过列表传入。
            如验证时取得了更好的验证结果(如果有多个Metric，以列表中第一个Metric为准)，且save_path不为None，
            则保存当前模型。Metric种类详见 :mod:`metrics模块 <fastNLP.core.metrics>` 。仅在传入dev_data时有效。
        :param str,None metric_key:  :class:`Metric<fastNLP.core.metrics.MetricBase>` 有时会有多个指标，
            比如 :class:`~fastNLP.core.metrics.SpanFPreRecMetric` 中包含了'f', 'pre', 'rec'。此时需
            要指定以哪个指标为准。另外有些指标是越小效果越好，比如语言模型的困惑度，这种情况下，在key前面增加一个'-'来表
            明验证时，值越小越好(比如: "-ppl")。仅在传入dev_data时有效。
        :param int validate_every: 多少个step在验证集上验证一次; 如果为-1，则每个epoch结束验证一次。仅在传入dev_data时有效。
        :param str,None save_path: 将模型保存路径，如果路径不存在，将自动创建文件夹。如果为None，则不保存模型。如果dev_data为None，则保存
            最后一次迭代的模型。保存的时候不仅保存了参数，还保存了模型结构。即便使用DataParallel，这里也只保存模型。
        :param bool use_tqdm: 是否使用tqdm来显示训练进度; 如果为False，则将loss打印在终端中。
        :param str,int,torch.device,list(int) device: 将模型load到哪个设备。默认为None，即Trainer不对模型
            的计算位置进行管理。支持以下的输入:
    
            1. str: ['cpu', 'cuda', 'cuda:0', 'cuda:1', ...] 依次为'cpu'中, 可见的第一个GPU中, 可见的第一个GPU中,
            可见的第二个GPU中;
    
            2. torch.device：将模型装载到torch.device上。
    
            3. int: 将使用device_id为该值的gpu进行训练
    
            4. list(int)：如果多于1个device，将使用torch.nn.DataParallel包裹model, 并使用传入的device。
    
            5. None. 为None则不对模型进行任何处理，如果传入的model为torch.nn.DataParallel该值必须为None。
    
            已知可能会出现的问题：Adagrad优化器可能无法正常使用这个参数，请手动管理模型位置。
    
        :param list(callbacks) callbacks: 用于在train过程中起调节作用的回调函数。比如early stop，negative sampling等可以
            通过callback机制实现。 可使用的callback参见 :mod:`callback模块 <fastNLP.core.callback>`
        :param int check_code_level: 模型检查等级. -1: 不进行检查; 0: 仅出现错误时停止; 1: 如果有field没有被使用，
            报告警告信息; 2: 有任何field没有被使用都报错. 检查的原理是通过使用很小的batch(默认2个sample)来运行代码，但是
            这个过程理论上不会修改任何参数，只是会检查能否运行。但如果(1)模型中存在将batch_size写为某个固定值的情况；
            (2)模型中存在累加前向计算次数的，可能会多计算1次。以上情况建议将check_code_level设置为-1。
        """
        super(Trainer, self).__init__()
        if not isinstance(model, nn.Module):
            raise TypeError(f"The type of model must be torch.nn.Module, got {type(model)}.")

        # check metrics and dev_data
        if (not metrics) and dev_data is not None:
            raise ValueError("No metric for dev_data evaluation.")
        if metrics and (dev_data is None):
            raise ValueError("No dev_data for evaluations, pass dev_data or set metrics to None. ")

        # check update every
        assert update_every >= 1, "update_every must be no less than 1."
        self.update_every = int(update_every)

        # check save_path
        if not (save_path is None or isinstance(save_path, str)):
            raise ValueError("save_path can only be None or `str`.")
        # prepare evaluate
        metrics = _prepare_metrics(metrics)

        # parse metric_key
        # increase_better is True. It means the exp result gets better if the indicator increases.
        # It is true by default.
        self.increase_better = True
        if metric_key is not None:
            self.increase_better = False if metric_key[0] == "-" else True
            self.metric_key = metric_key[1:] if metric_key[0] == "+" or metric_key[0] == "-" else metric_key
        else:
            self.metric_key = None
        # prepare loss
        losser = _prepare_losser(loss)

        if isinstance(train_data, BatchIter):
            if sampler is not None:
                warnings.warn("sampler is ignored when train_data is a BatchIter.")
            if num_workers>0:
                warnings.warn("num_workers is ignored when train_data is BatchIter.")
            if drop_last:
                warnings.warn("drop_last is ignored when train_data is BatchIter.")

        if isinstance(model, nn.parallel.DistributedDataParallel):  # 如果是分布式的
            # device为None
            if device is not None:
                warnings.warn("device is ignored when model is nn.parallel.DistributedDataParallel.")
                device = None
            # Sampler要是分布式的
            if sampler is None:
                sampler = torch.utils.data.DistributedSampler(train_data)
            elif not isinstance(sampler, torch.utils.data.DistributedSampler):
                raise TypeError("When using nn.parallel.DistributedDataParallel, "
                                "sampler must be None or torch.utils.data.DistributedSampler.")
            # 不能保存模型
            if save_path:
                raise RuntimeError("Saving model in Distributed situation is not allowed right now.")
        else:
            # sampler check
            if sampler is not None and not isinstance(sampler, (Sampler, torch.utils.data.Sampler)):
                raise ValueError(f"The type of sampler should be fastNLP.BaseSampler or pytorch's Sampler, got {type(sampler)}")
            if sampler is None:
                sampler = RandomSampler()
            elif hasattr(sampler, 'set_batch_size'):
                sampler.set_batch_size(batch_size)

        if isinstance(train_data, DataSet):
            self.data_iterator = DataSetIter(
                dataset=train_data, batch_size=batch_size, num_workers=num_workers, sampler=sampler, drop_last=drop_last)
        elif isinstance(train_data, BatchIter):
            self.data_iterator = train_data
            train_data = train_data.dataset
        else:
            raise TypeError("train_data type {} not support".format(type(train_data)))

        model.train()
        self.model = _move_model_to_device(model, device=device)
        if _model_contains_inner_module(self.model):
            self._forward_func = self.model.module.forward
        else:
            self._forward_func = self.model.forward
        if check_code_level > -1:
            # _check_code 是 fastNLP 帮助你检查代码是否正确的方法 。如果你在错误栈中看到这行注释，请认真检查你的field名与模型的输入
            #   名是否匹配
            dev_dataset = dev_data
            if isinstance(dev_data, BatchIter):
                dev_dataset = None
                warnings.warn("dev_data is of BatchIter type, ignore validation checking.")
            check_batch_size = min(batch_size, 1)
            if isinstance(self.model, nn.DataParallel):
                _num_devices = len(self.model.device_ids)
                if batch_size//_num_devices>1:  # 如果多卡是每个卡可以分多个数据的，则用每个卡给两个sample
                    check_batch_size = max(len(self.model.device_ids)*2, check_batch_size)
                else:
                    check_batch_size = max(len(self.model.device_ids), check_batch_size)
            _check_code(dataset=train_data, model=self.model, losser=losser, forward_func=self._forward_func, metrics=metrics,
                        dev_data=dev_dataset, metric_key=self.metric_key, check_level=check_code_level,
                        batch_size=check_batch_size)

        self.train_data = train_data
        self.dev_data = dev_data  # If None, No validation.
        self.losser = losser
        self.metrics = metrics
        self.n_epochs = int(n_epochs)
        self.batch_size = int(batch_size)
        self.save_path = save_path
        self.print_every = int(print_every)
        self.validate_every = int(validate_every) if validate_every != 0 else -1
        self.best_metric_indicator = None
        self.best_dev_epoch = None
        self.best_dev_step = None
        self.best_dev_perf = None
        self.n_steps = len(self.data_iterator) * self.n_epochs

        if isinstance(optimizer, torch.optim.Optimizer):
            self.optimizer = optimizer
        elif isinstance(optimizer, Optimizer):
            self.optimizer = optimizer.construct_from_pytorch(self.model.parameters())
        elif optimizer is None:
            self.optimizer = torch.optim.Adam(self.model.parameters(), lr=4e-3)
        else:
            raise TypeError("optimizer can only be torch.optim.Optimizer type, not {}.".format(type(optimizer)))

        self.logger = logger

        self.use_tqdm = use_tqdm
        if 'test_use_tqdm' in kwargs:
            self.test_use_tqdm = kwargs.get('test_use_tqdm')
        else:
            self.test_use_tqdm = self.use_tqdm
        self.pbar = None
        self.print_every = abs(self.print_every)
        self.kwargs = kwargs
        if self.dev_data is not None:
            self.tester = Tester(model=self.model,
                                 data=self.dev_data,
                                 metrics=self.metrics,
                                 batch_size=kwargs.get("dev_batch_size", self.batch_size),
                                 device=None,  # 由上面的部分处理device
                                 verbose=0,
                                 use_tqdm=self.test_use_tqdm)

        self.step = 0
        self.start_time = None  # start timestamp

        if isinstance(callbacks, Callback):
            callbacks = [callbacks]

        self.callback_manager = CallbackManager(env={"trainer": self},
                                                callbacks=callbacks)


    def train(self, load_best_model=True, on_exception='auto'):
        """
        使用该函数使Trainer开始训练。

        :param bool load_best_model: 该参数只有在初始化提供了dev_data的情况下有效，如果True, trainer将在返回之前重新加载dev表现
                最好的模型参数。
        :param str on_exception: 在训练过程遭遇exception，并被 :py:class:Callback 的on_exception()处理后，是否继续抛出异常。
                支持'ignore','raise', 'auto': 'ignore'将捕获异常，写在Trainer.train()后面的代码将继续运行; 'raise'将异常抛出;
                'auto'将ignore以下两种Exception: CallbackException与KeyboardInterrupt, raise其它exception.
        :return dict: 返回一个字典类型的数据,
                内含以下内容::

                    seconds: float, 表示训练时长
                    以下三个内容只有在提供了dev_data的情况下会有。
                    best_eval: Dict of Dict, 表示evaluation的结果。第一层的key为Metric的名称，
                                第二层的key为具体的Metric
                    best_epoch: int，在第几个epoch取得的最佳值
                    best_step: int, 在第几个step(batch)更新取得的最佳值

        """
        results = {}
        if self.n_epochs <= 0:
            self.logger.info(f"training epoch is {self.n_epochs}, nothing was done.")
            results['seconds'] = 0.
            return results
        try:
            self._model_device = _get_model_device(self.model)
            self._mode(self.model, is_test=False)
            self._load_best_model = load_best_model
            self.start_time = str(datetime.now().strftime('%Y-%m-%d-%H-%M-%S'))
            start_time = time.time()
            self.logger.info("training epochs started " + self.start_time)

            try:
                self.callback_manager.on_train_begin()
                self._train()
                self.callback_manager.on_train_end()

            except BaseException as e:
                self.callback_manager.on_exception(e)
                if on_exception == 'auto':
                    if not isinstance(e, (CallbackException, KeyboardInterrupt)):
                        raise e
                elif on_exception == 'raise':
                    raise e

            if self.dev_data is not None and self.best_dev_perf is not None:
                self.logger.info(
                    "\nIn Epoch:{}/Step:{}, got best dev performance:".format(self.best_dev_epoch, self.best_dev_step))
                self.logger.info(self.tester._format_eval_results(self.best_dev_perf))
                results['best_eval'] = self.best_dev_perf
                results['best_epoch'] = self.best_dev_epoch
                results['best_step'] = self.best_dev_step
                if load_best_model:
                    model_name = "best_" + "_".join([self.model.__class__.__name__, self.metric_key, self.start_time])
                    load_succeed = self._load_model(self.model, model_name)
                    if load_succeed:
                        self.logger.info("Reloaded the best model.")
                    else:
                        self.logger.info("Fail to reload best model.")
        finally:
            pass
        results['seconds'] = round(time.time() - start_time, 2)

        return results


    def _train(self):
        if not self.use_tqdm:
            from fastNLP.core.utils import _pseudo_tqdm as inner_tqdm
        else:
            inner_tqdm = tqdm
        self.step = 0
        self.epoch = 0
        start = time.time()
        with inner_tqdm(total=self.n_steps, postfix='loss:{0:<6.5f}', leave=False, dynamic_ncols=True) as pbar:
            self.pbar = pbar
            avg_loss = 0
            data_iterator = self.data_iterator
            self.batch_per_epoch = data_iterator.num_batches
            for epoch in range(1, self.n_epochs + 1):
                self.epoch = epoch
                pbar.set_description_str(desc="Epoch {}/{}".format(epoch, self.n_epochs))
                # early stopping
                self.callback_manager.on_epoch_begin()
                for batch_x, batch_y in data_iterator:
                    self.step += 1
                    _move_dict_value_to_device(batch_x, batch_y, device=self._model_device)
                    indices = data_iterator.get_batch_indices()
                    # negative sampling; replace unknown; re-weight batch_y
                    self.callback_manager.on_batch_begin(batch_x, batch_y, indices)
                    prediction = self._data_forward(self.model, batch_x)

                    # edit prediction
                    self.callback_manager.on_loss_begin(batch_y, prediction)
                    loss = self._compute_loss(prediction, batch_y).mean()
                    avg_loss += loss.item()
                    loss = loss / self.update_every

                    # Is loss NaN or inf? requires_grad = False
                    self.callback_manager.on_backward_begin(loss)
                    self._grad_backward(loss)
                    self.callback_manager.on_backward_end()

                    self._update()
                    self.callback_manager.on_step_end()

                    if self.step % self.print_every == 0:
                        avg_loss = float(avg_loss) / self.print_every
                        if self.use_tqdm:
                            print_output = "loss:{:<6.5f}".format(avg_loss)
                            pbar.update(self.print_every)
                        else:
                            end = time.time()
                            diff = timedelta(seconds=round(end - start))
                            print_output = "[epoch: {:>3} step: {:>4}] train loss: {:>4.6} time: {}".format(
                                epoch, self.step, avg_loss, diff)
                        pbar.set_postfix_str(print_output)
                        avg_loss = 0
                    self.callback_manager.on_batch_end()

                    if ((self.validate_every > 0 and self.step % self.validate_every == 0) or
                        (self.validate_every < 0 and self.step % len(data_iterator) == 0)) \
                            and self.dev_data is not None:
                        eval_res = self._do_validation(epoch=epoch, step=self.step)
                        eval_str = "Evaluation on dev at Epoch {}/{}. Step:{}/{}: ".format(epoch, self.n_epochs, self.step,
                                                                                    self.n_steps)
                        # pbar.write(eval_str + '\n')
                        self.logger.info(eval_str)
                        self.logger.info(self.tester._format_eval_results(eval_res)+'\n')
                # ================= mini-batch end ==================== #

                # lr decay; early stopping
                self.callback_manager.on_epoch_end()
            # =============== epochs end =================== #
            pbar.close()
            self.pbar = None
        # ============ tqdm end ============== #

    def _do_validation(self, epoch, step):
        self.callback_manager.on_valid_begin()
        res = self.tester.test()

        is_better_eval = False
        if self._better_eval_result(res):
            if self.save_path is not None:
                self._save_model(self.model,
                                 "best_" + "_".join([self.model.__class__.__name__, self.metric_key, self.start_time]))
            elif self._load_best_model:
                self._best_model_states = {name: param.cpu().clone() for name, param in self.model.state_dict().items()}
            self.best_dev_perf = res
            self.best_dev_epoch = epoch
            self.best_dev_step = step
            is_better_eval = True
        # get validation results; adjust optimizer
        self.callback_manager.on_valid_end(res, self.metric_key, self.optimizer, is_better_eval)
        return res

    def _mode(self, model, is_test=False):
        """Train mode or Test mode. This is for PyTorch currently.

        :param model: a PyTorch model
        :param bool is_test: whether in test mode or not.

        """
        if is_test:
            model.eval()
        else:
            model.train()

    def _update(self):
        """Perform weight update on a model.

        """
        if self.step % self.update_every == 0:
            self.optimizer.step()

    def _data_forward(self, network, x):
        x = _build_args(self._forward_func, **x)
        y = network(**x)
        if not isinstance(y, dict):
            raise TypeError(
                f"The return value of {_get_func_signature(self._forward_func)} should be dict, got {type(y)}.")
        return y

    def _grad_backward(self, loss):
        """Compute gradient with link rules.

        :param loss: a scalar where back-prop starts

        For PyTorch, just do "loss.backward()"
        """
        if (self.step-1) % self.update_every == 0:
            self.model.zero_grad()
        loss.backward()

    def _compute_loss(self, predict, truth):
        """Compute loss given prediction and ground truth.

        :param predict: prediction dict, produced by model.forward
        :param truth: ground truth dict, produced by batch_y
        :return: a scalar
        """
        return self.losser(predict, truth)

    def _save_model(self, model, model_name, only_param=False):
        """ 存储不含有显卡信息的state_dict或model
        :param model:
        :param model_name:
        :param only_param:
        :return:
        """
        if self.save_path is not None:
            model_path = os.path.join(self.save_path, model_name)
            if not os.path.exists(self.save_path):
                os.makedirs(self.save_path, exist_ok=True)
            if _model_contains_inner_module(model):
                model = model.module
            if only_param:
                state_dict = model.state_dict()
                for key in state_dict:
                    state_dict[key] = state_dict[key].cpu()
                torch.save(state_dict, model_path)
            else:
                model.cpu()
                torch.save(model, model_path)
                model.to(self._model_device)

    def _load_model(self, model, model_name, only_param=False):
        # 返回bool值指示是否成功reload模型
        if self.save_path is not None:
            model_path = os.path.join(self.save_path, model_name)
            if only_param:
                states = torch.load(model_path)
            else:
                states = torch.load(model_path).state_dict()
            if _model_contains_inner_module(model):
                model.module.load_state_dict(states)
            else:
                model.load_state_dict(states)
        elif hasattr(self, "_best_model_states"):
            model.load_state_dict(self._best_model_states)
        else:
            return False
        return True

    def _better_eval_result(self, metrics):
        """Check if the current epoch yields better validation results.

        :return bool value: True means current results on dev set is the best.
        """
        indicator, indicator_val = _check_eval_results(metrics, self.metric_key, self.metrics)
        if self.metric_key is None:
            self.metric_key = indicator
        is_better = True
        if self.best_metric_indicator is None:
            # first-time validation
            self.best_metric_indicator = indicator_val
        else:
            if self.increase_better is True:
                if indicator_val > self.best_metric_indicator:
                    self.best_metric_indicator = indicator_val
                else:
                    is_better = False
            else:
                if indicator_val < self.best_metric_indicator:
                    self.best_metric_indicator = indicator_val
                else:
                    is_better = False
        return is_better

    @property
    def is_master(self):
        """是否是主进程"""
        return True

In [386]:
# Модель TENER - такая же как и на гит https://github.com/fastnlp/TENER, но измененная для работы с моим классом embed

class TENER(nn.Module):
    def __init__(self, tag_vocab, embed, num_layers, d_model, n_head, feedforward_dim, dropout, encoding_type,
                 after_norm=True, attn_type='adatrans',  bi_embed=None,
                 fc_dropout=0.3, pos_embed=None, scale=False, dropout_attn=None):
        """

        :param tag_vocab: fastNLP Vocabulary
        :param embed: fastNLP TokenEmbedding
        :param num_layers: number of self-attention layers
        :param d_model: input size
        :param n_head: number of head
        :param feedforward_dim: the dimension of ffn
        :param dropout: dropout in self-attention
        :param after_norm: normalization place
        :param attn_type: adatrans, naive
        :param rel_pos_embed: position embedding
        :param bi_embed: Used in Chinese scenerio
        :param fc_dropout: dropout rate before the fc layer
        """
        super().__init__()

        self.embed = embed
        embed_size = self.embed.embed_size
        self.bi_embed = None
        if bi_embed is not None:
            self.bi_embed = bi_embed
            embed_size += self.bi_embed.embed_size

        self.in_fc = nn.Linear(embed_size, d_model)

        self.transformer = TransformerEncoder(num_layers, d_model, n_head, feedforward_dim, dropout,
                                              after_norm=after_norm, attn_type=attn_type,
                                              scale=scale, dropout_attn=dropout_attn,
                                              pos_embed=pos_embed)
        self.fc_dropout = nn.Dropout(fc_dropout)
        self.out_fc = nn.Linear(d_model, len(tag_vocab))
        
        
        self.crf = ConditionalRandomField(len(tag_vocab), include_start_end_trans=True, allowed_transitions=None)
        
    def _forward(self, chars, target, bigrams=None):
  
        # chars - Navec index
        mask = chars.ne(0)
        
        chars = self.embed.forward(chars)
        if self.bi_embed is not None:
            bigrams = self.bi_embed(bigrams)
            chars = torch.cat([chars, bigrams], dim=-1)
        
        chars = self.in_fc(chars)
        
        chars = self.transformer(chars, mask)
        chars = self.fc_dropout(chars)
        chars = self.out_fc(chars)
        logits = F.log_softmax(chars, dim=-1)
        if target is None:
            paths, _ = self.crf.viterbi_decode(logits, mask)
            return {'pred': paths}
        else:
            loss = self.crf(logits, target, mask)
            return {'loss': loss}

    def forward(self, chars, target, bigrams=None):
        return self._forward(chars, target, bigrams)

    def predict(self, chars, bigrams=None):
        
        return self._forward(chars, target=None, bigrams=bigrams)

In [387]:
# Определение параметров обучения и модели

if torch.cuda.is_available():
  device = torch.device('cuda')
else:
  device = None

normalize_embed = True
pos_embed = None
n_heads = 14 
head_dims = 128
num_layers = 2
lr = 0.000035
attn_type = 'adatrans'
char_type = 'cnn'

d_model = n_heads * head_dims 
dim_feedforward = int(2 * d_model)

batch_size = 2
warmup_steps = 0.01
dropout=0.15
fc_dropout=0.4
after_norm = 1
encoding_type = 'bioes'

save_path = '/content/'

In [388]:
model1 = TENER(tag_vocab=tar_for_embed, embed=embedding, num_layers=num_layers,
              d_model=d_model, n_head=n_heads,
              feedforward_dim=dim_feedforward, dropout=dropout,
              encoding_type=encoding_type,
              after_norm=after_norm, attn_type=attn_type,
              bi_embed=None, fc_dropout=fc_dropout,
              pos_embed=pos_embed, scale=attn_type=='transformer')

In [389]:
# Копия датасета для тренировки для оценки обучения
import copy
DT = copy.copy(dt_train)

DT.set_target('target')

+----------------+---------+--------------------+--------------------+--------------------+
| sentence       | seq_len | flag               | chars              | target             |
+----------------+---------+--------------------+--------------------+--------------------+
| ['государ...   | 2       | [True, True]       | [94791, 345483]... | [0, 0]             |
| ['республ...   | 5       | [True, True, Fa... | [376103, 176303... | [0, 0, 8, 0, 3]... |
| ['правопо...   | 4       | [False, True, T... | [329422, 51199,... | [8, 0, 0, 0]       |
| ['паспорт...   | 1       | [True]             | [288176]           | [0]                |
| ['государ...   | 5       | [True, True, Tr... | [94803, 345536,... | [0, 0, 0, 0, 8]... |
| ['и', 'обес... | 6       | [True, False, F... | [146473, 254239... | [0, 3, 8, 0, 0,... |
| ['далее', ...  | 2       | [True, True]       | [100318, 345483... | [0, 0]             |
| ['паспорт...   | 2       | [True, True]       | [288176, 310038... | [0, 0]   

In [390]:
# Инициализация Trainer
optimizer = optim.SGD(model1.parameters(), lr=lr, momentum=0.9)

callbacks = []
clip_callback = GradientClipCallback(clip_type='value', clip_value=5)
evaluate_callback = EvaluateCallback(data=DT) # Оценка train будет происходить на вкладке evaluation on test_data

if warmup_steps>0:
    warmup_callback = WarmupCallback(warmup_steps, schedule='linear')
    callbacks.append(warmup_callback)
callbacks.extend([clip_callback, evaluate_callback])

trainer = Trainer(dt_train, model1, optimizer, batch_size=30, sampler=BucketSampler(),
                  num_workers=2, n_epochs=100, dev_data=dt_test,
                  metrics=SpanFPreRecMetric(tag_vocab=tar_for_embed, encoding_type=encoding_type),
                  dev_batch_size=30, callbacks=callbacks, device=device, test_use_tqdm=False,
                  use_tqdm=True, print_every=300, save_path=save_path)

input fields after batch(if batch size is 1):
	chars: (1)type:torch.Tensor (2)dtype:torch.int64, (3)shape:torch.Size([1, 2]) 
	target: (1)type:torch.Tensor (2)dtype:torch.int64, (3)shape:torch.Size([1, 2]) 
target fields after batch(if batch size is 1):
	seq_len: (1)type:torch.Tensor (2)dtype:torch.int64, (3)shape:torch.Size([1]) 
	target: (1)type:torch.Tensor (2)dtype:torch.int64, (3)shape:torch.Size([1, 2]) 



In [ ]:
trainer.train(load_best_model=True)


training epochs started 2023-01-22-17-49-40


  0%|          | 0/1100 [00:00<?, ?it/s, loss:{0:<6.5f}]

Evaluate data in 32.63 seconds!
Evaluate data in 7.77 seconds!
EvaluateCallback evaluation on data-test:
SpanFPreRecMetric: f=0.0, pre=0.0, rec=0.0
Evaluation on dev at Epoch 1/100. Step:11/1100: 
SpanFPreRecMetric: f=0.0, pre=0.0, rec=0.0

Evaluate data in 32.39 seconds!
Evaluate data in 7.73 seconds!
EvaluateCallback evaluation on data-test:
SpanFPreRecMetric: f=0.0, pre=0.0, rec=0.0
Evaluation on dev at Epoch 2/100. Step:22/1100: 
SpanFPreRecMetric: f=0.0, pre=0.0, rec=0.0

Evaluate data in 32.57 seconds!
Evaluate data in 7.81 seconds!
EvaluateCallback evaluation on data-test:
SpanFPreRecMetric: f=0.0, pre=0.0, rec=0.0
Evaluation on dev at Epoch 3/100. Step:33/1100: 
SpanFPreRecMetric: f=0.069465, pre=0.347826, rec=0.038585

Evaluate data in 32.32 seconds!
Evaluate data in 10.18 seconds!
EvaluateCallback evaluation on data-test:
SpanFPreRecMetric: f=0.0, pre=0.0, rec=0.0
Evaluation on dev at Epoch 4/100. Step:44/1100: 
SpanFPreRecMetric: f=0.023548, pre=0.5, rec=0.012058

Evaluate d

{'best_eval': {'SpanFPreRecMetric': {'f': 0.345584,
   'pre': 0.544041,
   'rec': 0.253215}},
 'best_epoch': 35,
 'best_step': 385,
 'seconds': 3926.11}

In [ ]:
# Для проверки модели
for ch, y in DataSetIter(dt_train, batch_size=2):
  print(ch['chars'])
  mask = ch['chars'].ne(0)
  print(mask)
  out = model1.embed.forward(ch['chars'])
  print(out.shape)
  out = model1.in_fc(out)
  print(out.shape)
  out = model1.transformer(out, mask)
  print(out.shape)
  out = model1.fc_dropout(out)
  print(out.shape)
  out = model1.out_fc(out)
  print(out.shape)
  out = F.log_softmax(out, dim=-1)
  print(out.shape)
  paths, _ = model1.crf.viterbi_decode(out, mask)
  print(paths)
  break

tensor([[ 94791, 345483, 500001, 500001, 500001, 500001],
        [376103, 176303, 500000, 497635, 146473, 254239]])
tensor([[True, True, True, True, True, True],
        [True, True, True, True, True, True]])
torch.Size([2, 6, 330])
torch.Size([2, 6, 1792])
torch.Size([2, 6, 1792])
torch.Size([2, 6, 1792])
torch.Size([2, 6, 9])
torch.Size([2, 6, 9])
tensor([[0, 0, 0, 0, 0, 0],
        [0, 0, 0, 8, 0, 3]])
